<a href="https://colab.research.google.com/github/djalil21/sentiment-analysis-algerian-dialect/blob/master/DZ_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and corpus

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras-tuner


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import multiprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from kerastuner.tuners import Hyperband

<ipython-input-4-00f727a9ac47>:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband


In [ ]:
corpus = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PFE/all-data-preprocess-3.csv')

In [ ]:
corpus.head()

,sentiment,text
0,negative,سوء هاتف !
1,negative,عيوب لي في حمي زاف اتر عيا في في اميرا لي مليح...
2,negative,صراح ميصلحش شريت دم
3,negative,خاو سمحول صح هاتف 0 ثقيل ريزو
4,negative,خي هاتف را عند و سخ زاف اش نديرل


# preprocess

In [ ]:
corpus = corpus.drop('sentiment', axis=1)
#corpus = corpus.drop('id', axis=1)

In [ ]:
corpus = corpus.dropna(axis=0)

In [ ]:
corpus.text=corpus.text.astype(str)

In [ ]:
corpus.shape

(24865, 1)

In [ ]:
corpus.head()

,text
0,سوء هاتف !
1,عيوب لي في حمي زاف اتر عيا في في اميرا لي مليح...
2,صراح ميصلحش شريت دم
3,خاو سمحول صح هاتف 0 ثقيل ريزو
4,خي هاتف را عند و سخ زاف اش نديرل


#WORD2vec

In [ ]:
text = corpus['text'].apply(lambda x: x.split())

In [ ]:
Embedding_dimensions = 300

In [ ]:
cores = multiprocessing.cpu_count()
print(cores)

2


In [ ]:
# Defining the model and training it.
word2vec_model = Word2Vec(text,
                 vector_size=Embedding_dimensions,
                 workers=cores-1,
                 min_count=3)

In [ ]:
len(word2vec_model.wv)

9936

In [ ]:
word2vec_model.train(text, total_examples= len(text),epochs=40)

(9894986, 12757120)

In [ ]:
word2vec_model.save("/content/drive/MyDrive/Colab Notebooks/PFE/word2vec_model.bin")

# Classification DATA

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PFE/all-data-preprocess-3.csv')
#df =df.sample(n=5000)

In [ ]:
#df = df.drop('id', axis=1)

In [ ]:
df.text=df.text.astype(str)

In [ ]:
df = df[df.sentiment != "neutral"]

In [ ]:
df["sentiment"]=df["sentiment"].map({"negative":0,"positive":1})

<ipython-input-11-97d0437c83fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment"]=df["sentiment"].map({"negative":0,"positive":1})


In [ ]:
df.head()

,sentiment,text
0,0,سوء هاتف !
1,0,عيوب لي في حمي زاف اتر عيا في في اميرا لي مليح...
2,0,صراح ميصلحش شريت دم
3,0,خاو سمحول صح هاتف 0 ثقيل ريزو
4,0,خي هاتف را عند و سخ زاف اش نديرل


# Machine Learning

In [ ]:
word2vec_model = Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/PFE/word2vec_model.bin')

In [ ]:
sentences = df['text'].tolist()
labels = df['sentiment'].tolist()

In [ ]:
# Convert a sentence to a feature vector
def sentence_to_vec(sentence):
    vectors = []
    for word in sentence:
        if word in word2vec_model.wv:
            vectors.append(word2vec_model.wv[word])
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Convert all sentences to feature vectors
X = [sentence_to_vec(sentence) for sentence in sentences]

In [ ]:
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("DS1 svm acc ",accuracy_score(y_test, pred))

DS1 svm acc  0.7154975530179445


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("DS1 RandomForestClassifier acc ",accuracy_score(y_test, pred))

logreg = LogisticRegression()
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("DS1 LogisticRegression acc ",accuracy_score(y_test, pred))

clf = GaussianNB()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("DS1 GaussianNB acc ",accuracy_score(y_test, pred))

clf = BernoulliNB()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("DS1 BernoulliNB acc ",accuracy_score(y_test, pred))

DS1 RandomForestClassifier acc  0.7236541598694943


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


DS1 LogisticRegression acc  0.6858075040783034
DS1 GaussianNB acc  0.5138662316476346
DS1 BernoulliNB acc  0.5445350734094616



# Deep learning (not working)

In [ ]:
X_data, y_data = np.array(df['text']), np.array(df['sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,test_size = 0.2)

In [ ]:
"""
from gensim.models import Word2Vec

Embedding_dimensions = 300

# Creating Word2Vec training dataset.
Word2vec_train_data = list(map(lambda x: x.split(), X_train))
"""

'\nfrom gensim.models import Word2Vec\n\nEmbedding_dimensions = 300\n\n# Creating Word2Vec training dataset.\nWord2vec_train_data = list(map(lambda x: x.split(), X_train))\n'

In [ ]:
"""
import multiprocessing

cores = multiprocessing.cpu_count()
print(cores)
"""

'\nimport multiprocessing\n\ncores = multiprocessing.cpu_count()\nprint(cores)\n'

In [ ]:
"""
# Defining the model and training it.
word2vec_model = Word2Vec(Word2vec_train_data,
                 vector_size=Embedding_dimensions,
                 workers=cores-1,
                 min_count=5)

print("Vocabulary Length:", len(word2vec_model.wv.key_to_index))
"""

'\n# Defining the model and training it.\nword2vec_model = Word2Vec(Word2vec_train_data,\n                 vector_size=Embedding_dimensions,\n                 workers=cores-1,\n                 min_count=5)\n\nprint("Vocabulary Length:", len(word2vec_model.wv.key_to_index))\n'

In [ ]:
vocab_length = 60000
input_length = 300

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

Tokenizer vocab length: 60000


In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=input_length)
X_test  = pad_sequences(tokenizer.texts_to_sequences(X_test) , maxlen=input_length)

In [ ]:
print("X_train.shape:", X_train.shape)
print("X_test.shape :", X_test.shape)

X_train.shape: (39891, 300)
X_test.shape : (9973, 300)


In [ ]:
embedding_matrix = np.zeros((vocab_length+1, Embedding_dimensions))

for word, token in tokenizer.word_index.items():
    if word2vec_model.wv.__contains__(word):
        embedding_matrix[token] = word2vec_model.wv.__getitem__(word)

print("Embedding Matrix Shape:", embedding_matrix.shape)

IndexError: ignored

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding

In [ ]:
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")

    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = training_model.fit(
    X_train, y_train,
    batch_size=1024,
    epochs=12,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
acc,  val_acc  = history.history['accuracy'], history.history['val_accuracy']
loss, val_loss = history.history['loss'], history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
import seaborn as sns

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def ConfusionMatrix(y_pred, y_test):
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)

    categories  = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

In [ ]:
# Predicting on the Test dataset.
y_pred = training_model.predict(X_test)

# Converting prediction to reflect the sentiment predicted.
y_pred = np.where(y_pred>=0.5, 1, 0)

# Printing out the Evaluation metrics. 
ConfusionMatrix(y_pred, y_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(classification_report(y_test, y_pred))
print("DS1 BiLSTM acc ",accuracy_score(y_test, y_pred))

In [ ]:
import pickle

In [ ]:
# Saving Word2Vec-Model
word2vec_model.wv.save('Word2Vec-twitter-100')
word2vec_model.wv.save_word2vec_format('Word2Vec-twitter-100-trainable')

# Saving the tokenizer
with open('Tokenizer.pickle', 'wb') as file:
    pickle.dump(tokenizer, file)

# Saving the TF-Model.
training_model.save('Sentiment-BiLSTM')
training_model.save_weights("Model Weights/weights")

# Deep learning with Hyperband

In [ ]:
word2vec_model = Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/PFE/word2vec_model.bin')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense

In [ ]:
sentences = df['text'].tolist()
labels = df['sentiment'].tolist()

In [ ]:
# Tokenize and convert texts to sequences
sequences = []
for sentence in sentences:
    tokens = sentence.split()  # Assuming texts are already preprocessed
    sequence = [word2vec_model.wv.key_to_index[token] for token in tokens if token in word2vec_model.wv]
    sequences.append(sequence)

In [ ]:
# Pad sequences to a fixed length
max_sequence_length = 300
print(max_sequence_length)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

300


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
"""
# Define the BiLSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word2vec_model.wv), output_dim=100, input_length=max_sequence_length, trainable=False))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
"""

"""
# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=len(word2vec_model.wv), output_dim=100, input_length=max_sequence_length, trainable=False))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))
model.add(Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
"""

"\n# Define the model architecture\nmodel = Sequential()\nmodel.add(Embedding(input_dim=len(word2vec_model.wv), output_dim=100, input_length=max_sequence_length, trainable=False))\nmodel.add(Bidirectional(LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))\nmodel.add(Bidirectional(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2)))\nmodel.add(Dense(units=32, activation='relu'))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(1, activation='sigmoid'))\n"

In [ ]:
# Define the model-building function
def build_model(hp):
    # Define the hyperparameters to optimize
    learning_rate = hp.Float('learning_rate', 1e-5, 1e-2, sampling='log')
    num_hidden_units = hp.Choice('num_hidden_units', [32, 64, 128])
    dropout_rate = hp.Float('dropout_rate', 0.0, 0.5)
    batch_size = hp.Choice('batch_size', [16, 32, 64])
    
    # Build the BiLSTM model
    model = Sequential()
    model.add(Embedding(len(word2vec_model.wv), num_hidden_units, input_length=max_sequence_length))
    model.add(Bidirectional(LSTM(num_hidden_units, dropout=dropout_rate, return_sequences=True)))
    model.add(Bidirectional(LSTM(num_hidden_units, dropout=dropout_rate)))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

# Define the Hyperband tuner
tuner = Hyperband(build_model,
                  objective='val_accuracy',
                  max_epochs=5,
                  factor=3,
                  directory='/content/drive/MyDrive/Colab Notebooks/PFE/hyper',
                  project_name='sentiment_analysis')

# Perform hyperparameter search
tuner.search(X_train, y_train, validation_data=(X_test, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(1)[0]

"""
best_embedding_dim = best_hps.get('embedding_dim')
best_lstm_units = best_hps.get('lstm_units')
best_dense_units = best_hps.get('dense_units')
best_dropout = best_hps.get('dropout')
best_recurrent_dropout = best_hps.get('recurrent_dropout')
"""

Trial 10 Complete [00h 31m 33s]
val_accuracy: 0.8381729125976562

Best val_accuracy So Far: 0.8430668711662292
Total elapsed time: 07h 56m 21s


"\nbest_embedding_dim = best_hps.get('embedding_dim')\nbest_lstm_units = best_hps.get('lstm_units')\nbest_dense_units = best_hps.get('dense_units')\nbest_dropout = best_hps.get('dropout')\nbest_recurrent_dropout = best_hps.get('recurrent_dropout')\n"

In [ ]:
# Build and train the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 300, 128)          1271808   
                                                                 
 bidirectional_2 (Bidirectio  (None, 300, 256)         263168    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1,929,473
Trainable params: 1,929,473
Non-trainable params: 0
_________________________________________________________________


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = final_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, callbacks=[early_stopping])

Epoch 1/5
384/384 [==============================] - 1254s 3s/step - loss: 0.4309 - accuracy: 0.7975 - val_loss: 0.3598 - val_accuracy: 0.8437
Epoch 2/5
384/384 [==============================] - 1261s 3s/step - loss: 0.2623 - accuracy: 0.8922 - val_loss: 0.3971 - val_accuracy: 0.8411
Epoch 3/5
384/384 [==============================] - 1231s 3s/step - loss: 0.1807 - accuracy: 0.9257 - val_loss: 0.5150 - val_accuracy: 0.8434
Epoch 4/5
384/384 [==============================] - 1229s 3s/step - loss: 0.1481 - accuracy: 0.9403 - val_loss: 0.5046 - val_accuracy: 0.8385


In [ ]:
# Evaluate the final model
loss, accuracy = final_model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

96/96 [==============================] - 70s 719ms/step - loss: 0.3598 - accuracy: 0.8437
Test Loss: 0.35983985662460327
Test Accuracy: 0.8437194228172302
